# Train Stations in Europe analysis

In this notebook a statistical analysis on train stations in Europe will be made. The data contained in the dataset *train_stations_europe.csv* will be used to compute some statistics and to obtain some graphs.

## Dataset columns description:

- id: Numeric internal unique identifier. Primary key. <br /> <br />
- name: Name of the station as it is locally known. These names include accents and other special characters. <br /> <br />
- name_norm: Normalised version of name; transformed into [A-Za-z] character space (aka 'Latin-ASCII') to replace special characters with their standard-Latin counterparts (e.g. è become e, ü becomes u). <br /> <br />
- uic: The UIC code of the station. UIC is the International Union of Railways, "an international rail transport industry body". About 1/3 of all stations have no UIC code in this dataset. <br /> <br />
- longitude & latitude: Station coordinates. About 5% of all stations have no coordinates in this dataset. <br /> <br />
- parent_station_id: A station can belong to a meta station whose id is this value, i.e. Paris Gare d’Austerlitz (id = 4921) belongs to the meta-station Paris (id = 4916). About 92% of rows have NA entries. <br /> <br />
- country: Country codes in ISO 3166-1 alpha-2 format (2 digits). <br /> <br />
- time_zone: Continent/Country ISO codes. Those appear to be equivalent to Olson names (e.g. "Europe/Berlin"). <br /> <br />
- is_city: Marked as "unreliable" in the source dataset. Might be worth investigating what exactly that means. <br /> <br />
- is_main_station: Marked as "unreliable" in the source dataset. Might be worth investigating what exactly that means.

## Import libraries

We have to import some libraries which will be used in the code later:<br />
- numpy: perform powerful and efficient operation with numpy array <br />
- pandas: perform powerful and efficient operation on dataframe <br />
- math: perform mathematical operation <br />
- pyplot: perform data plots

In [73]:
import numpy as np
import pandas as pd                             
import math as mt
import matplotlib.pyplot as plt

## Read data
We first read the .csv file, then show the first elements in the dataset to see how it is built using pandas "head" function.

In [65]:
stations = pd.read_csv('train_stations_europe.csv', delimiter = ',')
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_row', None)
stations.head()

,id,name,name_norm,uic,latitude,longitude,parent_station_id,country,time_zone,is_city,is_main_station,is_airport,entur_id,entur_is_enabled
0,1,Château-Arnoux—St-Auban,Chateau-Arnoux-St-Auban,NaN,44.081790,6.001625,NaN,FR,Europe/Paris,True,False,False,NaN,False
1,2,Château-Arnoux—St-Auban,Chateau-Arnoux-St-Auban,8775123.0,44.061565,5.997373,1.0,FR,Europe/Paris,False,True,False,NaN,False
2,3,Château-Arnoux Mairie,Chateau-Arnoux Mairie,8775122.0,44.063863,6.011248,1.0,FR,Europe/Paris,False,False,False,NaN,False
3,4,Digne-les-Bains,Digne-les-Bains,NaN,44.350000,6.350000,NaN,FR,Europe/Paris,True,False,False,NaN,False
4,6,Digne-les-Bains,Digne-les-Bains,8775149.0,44.088710,6.222982,4.0,FR,Europe/Paris,False,True,False,NaN,False


## Clean data

I eliminate the last two columns _entur_id_, _entur_is_enabled_ because their meanings are not specified clearly. So they cannot be considered useful.

In [66]:
stations = stations.drop(['entur_id', 'entur_is_enabled'], axis = 1)
stations.head()

,id,name,name_norm,uic,latitude,longitude,parent_station_id,country,time_zone,is_city,is_main_station,is_airport
0,1,Château-Arnoux—St-Auban,Chateau-Arnoux-St-Auban,NaN,44.081790,6.001625,NaN,FR,Europe/Paris,True,False,False
1,2,Château-Arnoux—St-Auban,Chateau-Arnoux-St-Auban,8775123.0,44.061565,5.997373,1.0,FR,Europe/Paris,False,True,False
2,3,Château-Arnoux Mairie,Chateau-Arnoux Mairie,8775122.0,44.063863,6.011248,1.0,FR,Europe/Paris,False,False,False
3,4,Digne-les-Bains,Digne-les-Bains,NaN,44.350000,6.350000,NaN,FR,Europe/Paris,True,False,False
4,6,Digne-les-Bains,Digne-les-Bains,8775149.0,44.088710,6.222982,4.0,FR,Europe/Paris,False,True,False


Now I want to rewrite the _country_ columns, to clarify better what correspond to all the abbreviations (in the original dataset, countries are written as codes in ISO 3166-1 alpha-2 format 2 digits). To do this, I use another one dataset, in which there are matches between code and full name of the country.

In [67]:
iso_countries = pd.read_csv('iso_countries.csv', delimiter = ',')
iso_countries

,Name,Code
0,Afghanistan,AF
1,Åland Islands,AX
2,Albania,AL
3,Algeria,DZ
4,American Samoa,AS
...,...,...
244,Wallis and Futuna,WF
245,Western Sahara,EH
246,Yemen,YE
247,Zambia,ZM


At this point, a Merge between the two DataFrame _stations_ and _iso_countries_ is possible. 

In [68]:
iso_countries.rename(columns = {'Code':'country'}, inplace = True)

stations = pd.merge(stations, iso_countries, on = 'country')
stations.head()

,id,name,name_norm,uic,latitude,longitude,parent_station_id,country,time_zone,is_city,is_main_station,is_airport,Name
0,1,Château-Arnoux—St-Auban,Chateau-Arnoux-St-Auban,NaN,44.081790,6.001625,NaN,FR,Europe/Paris,True,False,False,France
1,2,Château-Arnoux—St-Auban,Chateau-Arnoux-St-Auban,8775123.0,44.061565,5.997373,1.0,FR,Europe/Paris,False,True,False,France
2,3,Château-Arnoux Mairie,Chateau-Arnoux Mairie,8775122.0,44.063863,6.011248,1.0,FR,Europe/Paris,False,False,False,France
3,4,Digne-les-Bains,Digne-les-Bains,NaN,44.350000,6.350000,NaN,FR,Europe/Paris,True,False,False,France
4,6,Digne-les-Bains,Digne-les-Bains,8775149.0,44.088710,6.222982,4.0,FR,Europe/Paris,False,True,False,France


In [72]:
stations = stations.drop(['country'], axis = 1)
stations.rename(columns = {'Name':'country'}, inplace = True)

# I reorder the columns in a specific way
columnsOrder = ['id', 'name', 'name_norm', 'latitude', 'longitude', 'country', 'uic', 
                'parent_station_id', 'time_zone', 'is_city', 'is_main_station', 'is_airport']
stations = stations.reindex(columns = columnsOrder)
stations.head()

,id,name,name_norm,latitude,longitude,country,uic,parent_station_id,time_zone,is_city,is_main_station,is_airport
0,1,Château-Arnoux—St-Auban,Chateau-Arnoux-St-Auban,44.081790,6.001625,NaN,NaN,NaN,Europe/Paris,True,False,False
1,2,Château-Arnoux—St-Auban,Chateau-Arnoux-St-Auban,44.061565,5.997373,NaN,8775123.0,1.0,Europe/Paris,False,True,False
2,3,Château-Arnoux Mairie,Chateau-Arnoux Mairie,44.063863,6.011248,NaN,8775122.0,1.0,Europe/Paris,False,False,False
3,4,Digne-les-Bains,Digne-les-Bains,44.350000,6.350000,NaN,NaN,NaN,Europe/Paris,True,False,False
4,6,Digne-les-Bains,Digne-les-Bains,44.088710,6.222982,NaN,8775149.0,4.0,Europe/Paris,False,True,False


<br />

# Data analysis

**Print total number of train stations in Europe**

In [62]:
print('Total number of European train stations collected into this dataset is', stations.name.count())

Total number of European train stations collected into this dataset is 63832


<br />

**Pivot table displaying the numbers of stations for each country**

In [63]:
stations_per_countries = stations.pivot_table('name', index = 'country', aggfunc = 'count')
stations_per_countries.rename(columns = {'name':'n° of stations'}, inplace = True)
display(stations_per_countries)

,n° of stations
country,
Albania,6
Andorra,10
Austria,1710
Belarus,20
Belgium,584
Bosnia and Herzegovina,131
Bulgaria,165
Croatia,705
Cyprus,8


In [78]:
#max(stations_per_countries.values)